In [1]:
import numpy as np
import math
from tqdm.notebook import tqdm
from functools import cache
import copy

import heapq



In [2]:
with open("input_day_20.txt") as f:
    text = f.read()


test = True
if test:
    text = r"""
broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a
"""

test2 = False
if test2:
    text = r"""
broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output
"""

text = text.strip()
#text = "button -> broadcaster\n" + text



In [3]:
states = {}
# broadcaster : (type, output_locs, just_received_low)
# flip-flop :   (type, output_locs, just_received_low, on_off)
# conjunction :  (type, output_locs, input_names, memories, just_received_pulse)

for line in text.split("\n"):
    name, outputs = line.split(" -> ")
    output_locs = [_.strip() for _ in outputs.split(",")]
    if name == "broadcaster":
        states["broadcaster"] = ["broadcaster", output_locs, "no"]
    elif name[0] == "%":
        states[name[1:]] = ["flip-flop", output_locs, "no", "off"]
    elif name[0] == "&":
        states[name[1:]] = ["conjunction", output_locs, [], [], "no"]

# filling in conjunction memories
for name, s in states.items():
    if s[0] == "conjunction":
        output_locs = s[1]
        input_names = []
        memories = []
        for name2, s2 in states.items():
            if name in s2[1]:
                input_names.append(name2)
                memories.append("low")
        states[name] = ["conjunction", output_locs, input_names, memories, "no"]

for k, v in states.items():
    print(k, v)

initial_states = copy.deepcopy(states)

broadcaster ['broadcaster', ['a', 'b', 'c'], 'no']
a ['flip-flop', ['b'], 'no', 'off']
b ['flip-flop', ['c'], 'no', 'off']
c ['flip-flop', ['inv'], 'no', 'off']
inv ['conjunction', ['a'], ['c'], ['low'], 'no']


In [4]:
detailed_print = False

def send_low_pulse(name_from, name_to):

    if detailed_print: print(name_from, "-low->", name_to)
    global states, num_low, num_high

    num_low += 1

    if not (name_to in states):
        return

    state_to = states[name_to]

    if state_to[0] == "broadcaster": # should never be the case
        assert False, "why outputting to broadcaster?!"
    
    elif state_to[0] == "flip-flop": # update received_low
        state_to[2] = "pending"

    elif state_to[0] == "conjunction": # update the memory
        state_to[4] = "pending"
        for i in range(len(state_to[2])):
            if state_to[2][i] == name_from:
                state_to[3][i] = "low"
    else:
        assert False, "weird connection!"

    states[name_to] = state_to # add the updates to states
    return


def send_high_pulse(name_from, name_to):
    if detailed_print: print(name_from, "-high->", name_to)

    global states, num_low, num_high

    num_high += 1

    if not (name_to in states):
        return

    state_to = states[name_to]

    if state_to[0] == "broadcaster": # should never be the case
        assert False, "why outputting to broadcaster?!"

    elif state_to[0] == "flip-flop": 
        state_to[2] = "no" # not sure about this, if it receives a low and a high pulse before getting updated, what should it do?
        return

    elif state_to[0] == "conjunction": # update the memory
        state_to[4] = "pending"
        for i in range(len(state_to[2])):
            if state_to[2][i] == name_from:
                state_to[3][i] = "high"
    else:
        assert False, "weird connection!"

    states[name_to] = state_to # add the updates to states
    return


def update():

    global states, num_low, num_high 

    for name, s in states.items():

        output_locs = s[1]
            
        if s[0] == "broadcaster":

            if s[2] == "yes": # just received low
                s[2] = "no"

                for name2 in output_locs: # loop through the outputs
                    send_low_pulse(name, name2)

        elif s[0] == "flip-flop":
            
            if s[2] == "yes": # received low pulse
                s[2] = "no" # clear input pulse
                if s[3] == "on": # was on, send a low pulse
                    s[3] = "off"

                    for name2 in output_locs: # loop through the outputs
                        send_low_pulse(name, name2)

                else: # was off, send a high pulse
                    s[3] = "on"

                    for name2 in output_locs: # loop through the outputs
                        send_high_pulse(name, name2)

        elif s[0] == "conjunction":

            if s[4] == "yes": # received pulse
                s[4] = "no"

                all_high = True
                for memory in s[3]:
                    if memory != "high":
                        all_high = False

                if all_high: # send a low pulse
                    for name2 in output_locs: # loop through the outputs
                        send_low_pulse(name, name2)

                else:
                    for name2 in output_locs: # loop through the outputs
                        send_high_pulse(name, name2)

        states[name] = s # apply changes to initial pulse

    for name, s in states.items():
        if s[0] == "broadcaster" and s[2] == "pending":
            s[2] = "yes"
        if s[0] == "flip-flop" and s[2] == "pending":
            s[2] = "yes"
        if s[0] == "conjunction" and s[4] == "pending":
            s[4] = "yes"

        states[name] = s

        

In [5]:
states = copy.deepcopy(initial_states)

detailed_print = False

num_low = 0
num_high = 0
for press_num in tqdm(range(1000)):

    #print(num_low, num_high)

    s = states["broadcaster"]
    s[2] = "yes"
    states["broadcaster"] = s

    num_low += 1 # button push
    
    if detailed_print: print("button -low-> broadcaster")

    while True:
        prev_states = copy.deepcopy(states)

        if detailed_print: 
            print("\nnew step:")
            for k, v in prev_states.items():
                print("   ", k, v)
        
        update()
        if states == prev_states:
            break

        

print(num_low, num_high, num_low * num_high)


  0%|          | 0/1000 [00:00<?, ?it/s]

8000 4000 32000000


In [6]:
# 634415985 too low

# 542848530, must also be too low